# Final model evaluation

This is the final notebook of the speed of sound prediction experiment.
It tests final models using three different tests.
 
 - General test
 - Inside matrix test
 - Outside matrix test

In [ ]:
# dependecies

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

__Restore the splitted data__

In [ ]:
%store -r X_train
%store -r y_train

In [ ]:
%store -r X_val
%store -r y_val

In [ ]:
%store -r X_test
%store -r y_test

__Merge training and validation data for general test on test data__

In [ ]:
X = pd.concat([X_train, X_val])
y = np.concatenate((y_train, y_val))

__Metrics funtion__

In [ ]:
# r2 and rmse and AARD returning function
from sklearn.metrics import r2_score, mean_squared_error

def result_stats(actual, predicted):
    """
    Returns r_2, rmse and AARD value for two arrays of equal length
    """
    
    r2 = r2_score(actual, predicted)
    rmse = np.sqrt(mean_squared_error( actual, predicted ))
    aard = (100 / len(actual)) * np.sum(np.abs((actual - predicted) / actual))
    
    return r2,rmse, aard

__Restore optimal hyperparameters, feature sets for SVM and random forest models__

In [ ]:
%store -r svm_params
%store -r svm_features

In [ ]:
%store -r rf_params
# the random forest has optimized max_features hyperparameter
# it is therefore trained on full feature set

__Import the models__

In [ ]:
from sklearn.svm import SVR as SVM
svm = SVM(gamma = "auto", **svm_params)    # gamma value is left at default; explicitly set to surpress warnings

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF
rf = RF(n_estimators = 20, **rf_params)    # use 20 estimators

## General test

This it the test the models were optimized for.

 - Train: development data (training + validation data)
 - Test: test data

__Plotting function__

In [ ]:
def general_test_plot(model_pred):
    """
    This function plots the residuals against both temperature and molality.
    It also plots predicted vs experimental values.
    Input: model predictions
    """
    
    # create figure for three graphs
    fig, ax = plt.subplots(ncols = 3, nrows = 1, figsize = (20, 5))

    
    # Residual plot against normalized temperature
    ax[0].scatter(x=X_test["T"], y=model_pred - y_test, s =0.5, c="r")
    ax[0].set_xlabel("$T$")
    ax[0].set_title("Residual plot against normalized temperature")
    ax[0].set_ylabel("$u$ [m/s]")
    
    # Residual plot against normalized molality
    ax[1].scatter(x=X_test["c"], y=model_pred - y_test, s =0.5, c="r")
    ax[1].set_xlabel("$c$")
    ax[1].set_title("Residual plot against normalized molality")
    ax[1].set_ylabel("$u$ [m/s]")
    
    # Predicted vs experimental values
    ax[2].scatter(y=y_test, x=model_pred, s =0.8, c ="r")
    ax[2].set_xlabel("Predicted $u$ [m/s]")
    ax[2].set_title("Predicted vs experimental values")
    ax[2].set_ylabel("Experimental $u$ [m/s]")
    
    # plot 0 residual line
    ax[2].set_ylim(ax[2].get_ylim()[0],ax[2].get_ylim()[1])
    ax[2].set_xlim(ax[2].get_xlim()[0],ax[2].get_xlim()[1])
    ax[2].plot([1000, 2000], [1000, 2000], linewidth = 0.2, c = "k")

### SVM regressor

__RMSE__

In [ ]:
# fit on development data, test on test set
# use correct feature set
svm.fit(X[svm_features], y)
svm_pred = svm.predict(X_test[svm_features])
print("General test RMSE for SVM model: RMSE {0:.2f}".format(result_stats(svm_pred,y_test)[1]))

__Result plots__

In [ ]:
general_test_plot(svm_pred)

# get current figure to add correct figure title
fig = plt.gcf()    
fig.suptitle("General test on SVM model", x = 0.5, y = 1, size = 14)
plt.show()

### Random forest regressor

__RMSE__

In [ ]:
# fit on development data, test on test set
rf.fit(X, y)
rf_pred = rf.predict(X_test)
print("General test RMSE for random forest model: RMSE {0:.2f}".format(result_stats(rf_pred,y_test)[1]))

__Result plots__

In [ ]:
general_test_plot(rf_pred)

# get current figure to add correct figure title
fig = plt.gcf()    
fig.suptitle("General test on random forest model", x = 0.5, y = 1, size = 14)
plt.show()

## Inside matrix test

This test tests how well the models can predict data for an electrolyte not present in training data.<br>
For each medium a graph of both predicted (red) and actual (black) speed of sound values against temperature is plotted.

 - Train: data on all but one electrolyte
 - Test: the remaining electrolyte data

In [ ]:
# load dataset with all the preprocessed data
# the original temperature values are stored in T_orig feature
df = pd.read_csv("../datasets/preprocessed.csv", index_col = 0)

In [ ]:
df.head()

In [ ]:
# get the list of all available electrolytes
mediums = df["medium"].unique()

### Run the inside matrix test for each electrolyte

The inside_matrix_test function defined below plots the graph of inside matrix test for each electrolyte. The RMSE error is noted in the graph's title.

In [ ]:
def inside_matrix_test(model, features):
    """
    This function plots actual and predicted speed of sound values for all mediums in df.
    df is the dataframe with all data.
    The predictions are trained on input model with input feature set.
    """
    for medium in mediums:
        # form the test/train sets
        df_test = df[df["medium"] == medium]
        df_train = df[df["medium"] != medium]
    
        model.fit(df_train[features],df_train["sound"])
        model_pred = model.predict(df_test[features])
        _, r, _ = result_stats(model_pred,df_test["sound"])
        
        # plot predictions red
        plt.scatter(x=df_test["T_orig"], y=model_pred, s =0.5, c = "r")        
        # plot actual values black
        plt.scatter(x=df_test["T_orig"], y=df_test["sound"], s =0.5, c="k")
    
        plt.xlabel("$T$ [K]")
        plt.title("Predicted vs real speed of sound values of {0}. RMSE: {1:.1f}".format(medium, r))
        plt.ylabel("$u$ [m/s]")

        plt.show()

### SVM regressor

In [ ]:
inside_matrix_test(svm, svm_features)

### Random forest regressor

In [ ]:
inside_matrix_test(rf, list(X.columns))

### List of all inside matrix test results

In [ ]:
for medium in mediums:
    
    df_test = df[df["medium"] == medium]
    df_train = df[df["medium"] != medium]
    
    # svm predictions
    svm.fit(df_train[svm_features],df_train["sound"])
    svm_pred = svm.predict(df_test[svm_features])
    _, r_svm , _ = result_stats(svm_pred,df_test["sound"])
    
    # rf predictions
    rf.fit(df_train[list(X.columns)],df_train["sound"])
    rf_pred = rf.predict(df_test[list(X.columns)])
    _, r_rf , _ = result_stats(rf_pred,df_test["sound"])
    
    print("{0:10} SVM RMSE {1:5.1f}; RF RMSE {2:5.1f}".format(medium, r_svm, r_rf))

## Outside matrix test

This test tests how well the models can predict data for an electrolyte having of its ions completely outside the training data.

 - Train: all (training + validation + test data)
 - Test: additional RbI data

In [ ]:
# load the RbI dataset
try:
    outtest = pd.read_csv("../datasets/o_preprocessed.csv", index_col = 0)    
except FileNotFoundError:
        print("The creation of o_preprocessed.csv must have been skipped")

### SVM regressor

__RMSE__

In [ ]:
svm.fit(df[svm_features], df["sound"])
svm_pred = svm.predict(outtest[svm_features])
print("Outside matrix test RMSE for SVM model: RMSE {0:.2f}".format(result_stats(svm_pred,outtest["sound"])[1]))

__Plot of predicted (red) and experimental (black) sound speeds__

In [ ]:
plt.scatter(x=outtest["T"], y=svm_pred, marker = "o", s =0.5, c = "r")
plt.scatter(x=outtest["T"], y=outtest["sound"], s =0.5, c="k", marker = 'x')

plt.xlabel("$T$ [K]")
plt.title("Predicted vs real speed of sound values of RbI")
plt.ylabel("$u$ [m/s]")

plt.show()

### Random forest regressor

__RMSE__

In [ ]:
rf.fit(df[list(X.columns)], df["sound"])
rf_pred = rf.predict(outtest[list(X.columns)])
print("Outside matrix test RMSE for random forest model: RMSE {0:.2f}".format(result_stats(rf_pred,outtest["sound"])[1]))

__Plot of predicted (red) and experimental (black) sound speeds__

In [ ]:
plt.scatter(x=outtest["T"], y=rf_pred, marker = "o", s =0.5, c = "r")
plt.scatter(x=outtest["T"], y=outtest["sound"], s =0.5, c="k", marker = 'x')

plt.xlabel("$T$ [K]")
plt.title("Predicted vs real speed of sound values of RbI")
plt.ylabel("$u$ [m/s]")

plt.show()

### Variation on outside matrix test
__This is not discussed in the thesis__<br>
Variation on the outside matrix test was carried out by removing all data for a fixed cation or anion from the original electrolytes. This test was carried for each ion.

 - Train: All data, except for all data for a given ion
 - Test: All data for the removed ion

__Define all ions__

In [ ]:
# lists storing used ionic species

cations = ["Na", "K", "Li", "NH4", "H"]
anions = ["Br","Cl", "I", "2SO4", "2CO3", "NO3", "OH"]

__Outside matrix test for specific cation__

In [ ]:
for c in cations:
    # put data for cation c into test set
    df_test = df[df["medium"].apply(lambda x: x.startswith(c))]
    df_train = df[df["medium"].apply(lambda x: not x.startswith(c))]
    
    # svm predictions
    svm.fit(df_train[svm_features],df_train["sound"])
    svm_pred = svm.predict(df_test[svm_features])
    _, r_svm , _ = result_stats(svm_pred,df_test["sound"])
    
    # rf predictions
    rf.fit(df_train[list(X.columns)],df_train["sound"])
    rf_pred = rf.predict(df_test[list(X.columns)])
    _, r_rf , _ = result_stats(rf_pred,df_test["sound"])
    
    print("{0}: SVM RMSE {1:.1f}; RF RMSE {2:.1f}".format(c, r_svm, r_rf))

__Outside matrix test for specific anion__

In [ ]:
for a in anions:
    # put data for anion a into test set
    df_test = df[df["medium"].apply(lambda x: x.endswith(a))]
    df_train = df[df["medium"].apply(lambda x: not x.endswith(a))]
    
    # svm predictions
    svm.fit(df_train[svm_features],df_train["sound"])
    svm_pred = svm.predict(df_test[svm_features])
    _, r_svm , _ = result_stats(svm_pred,df_test["sound"])
    
    # rf predictions
    rf.fit(df_train[list(X.columns)],df_train["sound"])
    rf_pred = rf.predict(df_test[list(X.columns)])
    _, r_rf , _ = result_stats(rf_pred,df_test["sound"])
    
    print("{0}: SVM RMSE {1:.1f}; RF RMSE {2:.1f}".format(a, r_svm, r_rf))